In [18]:
import pandas as pd

team_name_mapping = {
    "Arizona Cardinals": "ARI",
    "Atlanta Falcons": "ATL",
    "Baltimore Ravens": "BAL",
    "Buffalo Bills": "BUF",
    "Carolina Panthers": "CAR",
    "Chicago Bears": "CHI",
    "Cincinnati Bengals": "CIN",
    "Cleveland Browns": "CLE",
    "Dallas Cowboys": "DAL",
    "Denver Broncos": "DEN",
    "Detroit Lions": "DET",
    "Green Bay Packers": "GB",
    "Houston Texans": "HOU",
    "Indianapolis Colts": "IND",
    "Jacksonville Jaguars": "JAX",
    "Kansas City Chiefs": "KC",
    "Miami Dolphins": "MIA",
    "Minnesota Vikings": "MIN",
    "New England Patriots": "NE",
    "New Orleans Saints": "NO",
    "New York Giants": "NYG",
    "New York Jets": "NYJ",
    "Oakland Raiders": "LV",  # Fixed
    "Philadelphia Eagles": "PHI",
    "Pittsburgh Steelers": "PIT",
    "San Diego Chargers": "LAC",
    "San Francisco 49ers": "SF",
    "Seattle Seahawks": "SEA",
    "St. Louis Rams": "LAR",  # Fixed
    "Tampa Bay Buccaneers": "TB",
    "Tennessee Titans": "TEN",
    "Washington Football Team": "WAS"  # Fixed
}

# Function to replace team names in the DataFrame
def replace_team_names(df, mapping):
    df['Team'] = df['Team'].map(mapping)
    return df

# File paths
files = {
    "Half PPR": "/Users/kmaran3/Dropbox/Darkhorse/Final Rankings/Half PPR Rankings.csv",
    "Full PPR": "/Users/kmaran3/Dropbox/Darkhorse/Final Rankings/Full PPR Rankings.csv",
    "Non PPR": "/Users/kmaran3/Dropbox/Darkhorse/Final Rankings/Non PPR Rankings.csv"
}

# Define weights
vbd_weight = 0.3
adp_weight = 0.7

# Loop through each file
for key, file in files.items():
    # Load the CSV file
    df = pd.read_csv(file)
    
    # Get the baseline points for each position
    baseline_qb = df[(df['Position'] == 'QB')]['Final PPG'].iloc[min(11, len(df[df['Position'] == 'QB'])-1)]
    baseline_rb = df[(df['Position'] == 'RB')]['Final PPG'].iloc[min(23, len(df[df['Position'] == 'RB'])-1)]
    baseline_wr = df[(df['Position'] == 'WR')]['Final PPG'].iloc[min(29, len(df[df['Position'] == 'WR'])-1)]
    baseline_te = df[(df['Position'] == 'TE')]['Final PPG'].iloc[min(11, len(df[df['Position'] == 'TE'])-1)]
    
    # Calculate VBD for each player based on their position
    def calculate_vbd(row):
        if row['Position'] == 'QB':
            return row['Final PPG'] - baseline_qb
        elif row['Position'] == 'RB':
            return row['Final PPG'] - baseline_rb
        elif row['Position'] == 'WR':
            return row['Final PPG'] - baseline_wr
        elif row['Position'] == 'TE':
            return row['Final PPG'] - baseline_te
        else:
            return 0
    
    df['VBD'] = df.apply(calculate_vbd, axis=1)
    
    # Normalize VBD and ESPN ADP
    df['Normalized VBD'] = (df['VBD'] - df['VBD'].min()) / (df['VBD'].max() - df['VBD'].min())
    df['Normalized ADP'] = (df['ESPN ADP'] - df['ESPN ADP'].min()) / (df['ESPN ADP'].max() - df['ESPN ADP'].min())
    
    # Calculate the weighted score
    df['Weighted Score'] = (vbd_weight * df['Normalized VBD']) + (adp_weight * (1 - df['Normalized ADP']))
    
    # Sort by weighted score in descending order to get the top players based on the weighted score
    df = df.sort_values(by='Weighted Score', ascending=False)
    
    # Adjust the ranking column
    df['Rank'] = range(1, len(df) + 1)

    df = replace_team_names(df, team_name_mapping)

    df['Position Rank'] = df.groupby('Position').cumcount() + 1
    df['Position'] = df['Position'] + df['Position Rank'].astype(str)
    df.drop(columns=['Position Rank'], inplace=True)
    
    # Save the updated DataFrame to a new CSV file
    output_file = file.replace(".csv", " with Weighted VBD.csv")
    df.to_csv(output_file, index=False)

df


,Unnamed: 0,Rank,Name,Team,Position,Final PPG,Bye Week,ESPN ADP,VBD,Normalized VBD,Normalized ADP,Weighted Score
34,34,1,Breece Hall,NYJ,RB1,11.684624,12,4.0,3.999925,0.950448,0.012048,0.976701
33,33,2,Bijan Robinson,ATL,RB2,11.784514,12,5.0,4.099814,0.954849,0.016064,0.975210
31,31,3,Jahmyr Gibbs,DET,RB3,12.809350,5,14.0,5.124650,1.000000,0.052209,0.963454
32,32,4,Kyren Williams,LAR,RB4,12.390651,6,15.0,4.705951,0.981553,0.056225,0.955109
45,45,5,Christian McCaffrey,SF,RB5,9.382637,9,1.0,1.697938,0.849029,0.000000,0.954709
...,...,...,...,...,...,...,...,...,...,...,...,...
453,453,454,Donald Parham,LAC,TE99,1.887786,5,NaN,-1.748556,0.697186,NaN,NaN
454,454,455,Juwan Johnson,NO,WR180,1.808783,12,NaN,-2.707143,0.654954,NaN,NaN
455,455,456,Matt Barkley,JAX,QB69,1.706371,12,NaN,-16.207383,0.060171,NaN,NaN
456,456,457,Cooper Rush,DAL,QB70,1.576184,7,NaN,-16.337570,0.054436,NaN,NaN
